# Importing packages

In [1]:
from workers import MasterNode
from models import LinReg, LogReg, LogRegNoncvx, NN_1d_regression
from utils import read_run, get_alg, create_plot_dir, PLOT_PATH
from sklearn.datasets import dump_svmlight_file

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from prep_data import number_of_features
import math
import torch

from numpy.random import default_rng
from numpy import linalg as la
from prep_data import DATASET_PATH
import copy
import pickle

# Customizing Matplotlib

In [2]:
plt.style.use('fast')
mpl.rcParams['mathtext.fontset'] = 'cm'
# mpl.rcParams['mathtext.fontset'] = 'dejavusans'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['lines.linewidth'] = 2.0
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

In [3]:
markers = ['x', '.', '+', '1', 'p','*', 'D' , '.',  's']

In [4]:
MODELS_PATH = 'models/'

# Experiment 1. Can weak models collaboratively train one good global model?

## Medium size dataset

In [ ]:
number_of_tasks = 50
number_of_points_per_task = 30
dataset_name = 'artificial_sine_medium'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
for i in range(number_of_tasks):
    a = 0.1 + rng.random() * (2.0 - 0.1)
    a_array.append(a)
    b = rng.random() * 2 * np.pi
    b_array.append(b)
    x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
    y_train = a * np.sin(x_train + b)
    X_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = x_train.copy()
    y_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = y_train.squeeze(1).copy()
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
for i in range(min(5, number_of_tasks)):
    inds = range(number_of_points_per_task * i, number_of_points_per_task * (i + 1))
    plt.scatter(X_sine[inds], y_sine[inds])

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)

In [ ]:
key_word = '_medium'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=0.1)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(10):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.arange(0, 1.0, 0.1)

In [ ]:
for i in range(1,10):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].w_opt_global = np.zeros(models[0].d)
models[0].change_alpha(0.0)

In [ ]:
for i in range(2, 10):
    model = models[i]
    min_L = 1e-6
    max_L = 0.1
    max_it = 100000
    tol = 0.1
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                  break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        print('')
        print('Unknown problem')

In [ ]:
MODELS_PATH = 'models'

In [ ]:
models_dict = {'models' : models}

In [ ]:
with open('models_medium', 'wb') as file:
    pickle.dump(models_dict, file)

### Performance on existing clients

In [ ]:
mse_table = np.empty(shape=(5, models[0].n_workers))

In [ ]:
models[0].change_alpha(0.0)

In [ ]:
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
for i in range(10):
    print(models[i].workers[0].alpha)

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(5):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
mse_alpha.shape

In [ ]:
plt.plot(mse_alpha)

### Performance on a new client

In [ ]:
n_train = 30
n_test = 1000

In [ ]:
rng = default_rng()
a_cn = 0.1 + rng.random() * (2.0 - 0.1)
b_cn = rng.random() * 2 * np.pi
x_train_cn = -5.0 + rng.random((n_train, 1)) * 10.0
y_train_cn = a_cn * np.sin(x_train_cn + b_cn)

In [ ]:
plt.scatter(x_train_cn, y_train_cn)

In [ ]:
control_node = NN_1d_regression(id_node=1000, alpha=0.5, x_train=x_train_cn, y_train=y_train_cn, regularization=None, tolerance=0.1)

In [ ]:
control_node.set_weights(control_node.w_opt)

In [ ]:
y_pred_train = control_node.model(torch.from_numpy(x_train_cn).float()).detach().numpy()

In [ ]:
plt.scatter(x_train_cn, y_pred_train)

In [ ]:
x_test_cn = -5.0 + rng.random((n_test, 1)) * 10.0
y_test_cn = a_cn * np.sin(x_test_cn + b_cn)

In [ ]:
plt.scatter(x_test_cn, y_test_cn)

In [ ]:
mse_array = np.empty(5)

In [ ]:
for i in range(5):
    control_node.alpha = 0.1 * i
    control_node.set_weights(control_node.compute_local(models[i].w_opt_global))
    y_prediction_cn = control_node.model(torch.from_numpy(x_test_cn).float()).detach().numpy()
    mse_array[i] = np.mean((y_prediction_cn - y_test_cn) ** 2)

In [ ]:
mse_array

Unfortunately, at least in the checked set-ups weak models do not give a good global model.

# Experiment 2. How much do few gradient steps help the mixed model (if it helps at all)?

In [ ]:
with open(MODELS_PATH + 'models_extended', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
models[0].n_workers

Precision to which each worker is trained to.

In [ ]:
for i in range(models[0].n_workers):
    curr_worker = models[0].workers[i]
    print(i, curr_worker.fun_value(curr_worker.w_opt))

Precision to which each global model is trained to.

In [ ]:
for model in models:
    print(model.fun_value(model.w_opt_global))

The number of workers is large. The higher alpha is, the harder is to train the global model.

In [ ]:
models[0].alpha = np.zeros(models[0].n_workers)

In [ ]:
def few_gradient_steps(test_node, trained_global_model, num_grad_steps, stepsize):
    assert np.std(trained_global_model.alpha) < 1e-5
    test_node.alpha = trained_global_model.alpha[0]
    w = copy.deepcopy(trained_global_model.w_opt_global)
    for step in range(num_grad_steps):
        grad = test_node.grad_shift(test_node.x_train, test_node.y_train, w)
        w -= stepsize * grad
    return w

In [ ]:
number_of_points = 50
rng = default_rng()

In [ ]:
a = 0.1 + rng.random() * (2.0 - 0.1)
b = rng.random() * 2 * np.pi
x_train = -5.0 + rng.random((number_of_points, 1)) * 10.0
y_train = a * np.sin(x_train + b)
test_node = NN_1d_regression(id_node=1000, alpha=0.1, x_train=x_train, y_train=y_train, regularization=None, tolerance=1e-2)

In [ ]:
w = few_gradient_steps(test_node, models[1], 5, 1e-3)

In [ ]:
number_of_experiments = 1
number_of_train_points = 50
number_of_test_points = 1000
grad_steps = [0, 5, 10, 15]
rng = default_rng()
mse_table = np.zeros(shape=(len(models), len(grad_steps)))
for ind in range(number_of_experiments):
    a = 0.1 + rng.random() * (2.0 - 0.1)
    b = rng.random() * 2 * np.pi
    x_train = -5.0 + rng.random((number_of_train_points, 1)) * 10.0
    y_train = a * np.sin(x_train + b)
    x_test = -5.0 + rng.random((number_of_train_points, 1)) * 10.0
    y_test = a * np.sin(x_test + b)
    test_node = NN_1d_regression(id_node=1000, alpha=0.1, x_train=x_train, y_train=y_train, regularization=None, tolerance=1e-2)
    ind_1 = 0
    for model in models:
        ind_2 = 0
        for num_grad_steps in grad_steps:
            w = few_gradient_steps(test_node, model, num_grad_steps, 1e-3)
            test_node.set_weights(test_node.compute_local(w))
            y_predicted = test_node.model(torch.from_numpy(x_test).float()).detach().numpy()
            mse_table[ind_1][ind_2] += np.mean((y_predicted - y_test) ** 2) / number_of_experiments
            ind_2 += 1
        ind_1 += 1

In [ ]:
mse_table

# Experiment 3. Constant a, large number of tasks

## New dataset

In [ ]:
number_of_tasks = 1000
number_of_points_per_task = 20
dataset_name = 'artificial_sine_a_constant'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))

In [ ]:
rng = default_rng()
b_array = []
for i in range(number_of_tasks):
    b = rng.random() * 2 * np.pi
    b_array.append(b)
    x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
    y_train = np.sin(x_train + b)
    X_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = x_train.copy()
    y_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = y_train.squeeze(1).copy()
b_array = np.array(b_array)

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)

In [ ]:
np.save(DATASET_PATH + 'b_constant.npy', b_array)

## Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 1

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=0.1)
print('Running GD...')
model.run_gd(max_it)

## Training global models

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.arange(0.9, 1.01, 0.01, dtype=np.float)

In [ ]:
alphas

In [ ]:
for i in range(0,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models_dict = {'models' : models}

In [ ]:
with open(MODELS_PATH + 'a_constant_1000', 'wb') as file:
    pickle.dump(models_dict, file)

In [ ]:
# models[0].change_alpha(0.0)
# models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
2+2

In [ ]:
for i in range(0, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 100000
    tol = 0.1
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global) if i != 0 else np.zeros(models[0].d)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models)
        print('')
        print('Unknown problem')

## Control node

In [ ]:
numer_of_test_tasks = 25
for i in range(numer_of_test_tasks):
    b = rng.random() * 2 * np.pi
    x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
    y_train = np.sin(x_train + b)
    x_test =  -5.0 + rng.random((1000, 1)) * 10.0
    y_test = np.sin(x_test + b)
    tolerance = 0.1
    test_node = NN_1d_regression(id_node=1000, alpha=0.1, x_train=x_train, y_train=y_train, regularization=None, tolerance=tolerance)
    test_node.set_weights(test_node.w_opt)
    y_predicted = test_node.model(torch.from_numpy(x_test).float()).detach().numpy()
    print('Alpha = 0   {}'.format(np.mean((y_predicted - y_test) ** 2)))
    
    test_node.set_weights(test_node.compute_local(models[1].w_opt_global))
    y_predicted = test_node.model(torch.from_numpy(x_test).float()).detach().numpy()
    print('Alpha = 0.1 {}'.format(np.mean((y_predicted - y_test) ** 2)))

## Experiment 4. Basic sine experiment

In [ ]:
number_of_tasks = 100
number_of_points_per_task = 50
dataset_name = 'artificial_sine_50_100'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
rng = default_rng()
a_array = []
b_array = []
for i in range(number_of_tasks):
    a = 0.1 + rng.random() * (5.0 - 0.1)
    a_array.append(a)
    b = rng.random() * 2 * np.pi
    b_array.append(b)
    x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
    y_train = a * np.sin(x_train + b)
    X_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = x_train.copy()
    y_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = y_train.squeeze(1).copy()
a_array = np.array(a_array)
b_array = np.array(b_array)
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
np.save(DATASET_PATH + 'a_50_100.npy', a_array)
np.save(DATASET_PATH + 'b_50_100.npy', b_array)

In [ ]:
alpha = 0.5
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2)
print('Running GD...')
model.run_gd(max_it)

In [ ]:
with open(MODELS_PATH + 'model_initial_50_100', 'wb') as file:
    model_init_dict = {'model' : model}
    pickle.load(model_init_dict, file)

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.arange(0, 1.1, 0.1)

In [ ]:
alphas

In [ ]:
for i in range(0, 11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
for i in range(2, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 100000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, '50_100')
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, '50_100')
        print('')
        print('Unknown problem')

In [ ]:
def save(models, filename):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
save(models, '50_100')

In [ ]:
with open(MODELS_PATH + '50_100', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
# a_array = np.load(DATASET_PATH +'a_50_100.npy')
# b_array = np.load(DATASET_PATH + 'b_50_100.npy')

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymax=mse_alpha[argmin])
plt.tight_layout()
# plt.savefig(PLOT_PATH + '/50_100_MSE_over_alphas.pdf')

## 5. Basic sine experiment with validation criterion stop

In [ ]:
number_of_tasks = 100
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 10
dataset_name = 'artificial_sine_50_100'
validation_dataset_name = 'artificial_sine_10_100_validation'

In [ ]:
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))
rng = default_rng()
a_array = np.load(DATASET_PATH + 'a_50_100.npy')
b_array = np.load(DATASET_PATH + 'b_50_100.npy')
for i in range(number_of_tasks):
    x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
    y_val = a_array[i] * np.sin(x_val + b_array[i])
    X_sine_val[number_of_points_per_task_for_validation * i : number_of_points_per_task_for_validation * (i + 1)] = x_val.copy()
    y_sine_val[number_of_points_per_task_for_validation * i : number_of_points_per_task_for_validation * (i + 1)] = y_val.squeeze(1).copy()
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
alpha = 0.5
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

In [ ]:
with open(MODELS_PATH + 'model_initial_50_100_validated_10', 'wb') as file:
    model_init_dict = {'model' : model}
    pickle.load(model_init_dict, file)

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.arange(0, 1.1, 0.1)

In [ ]:
alphas

In [ ]:
for i in range(0, 11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
for i in range(1, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, '50_100_validated')
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, '50_100_validated')
        print('')
        print('Unknown problem')

In [ ]:
def save(models, filename):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
save(models, '50_100')

In [ ]:
with open(MODELS_PATH + '50_100_validated', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
# a_array = np.load(DATASET_PATH +'a_50_100.npy')
# b_array = np.load(DATASET_PATH + 'b_50_100.npy')

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
a_array = np.load(DATASET_PATH + 'a_50_100.npy')
b_array = np.load(DATASET_PATH + 'b_50_100.npy')

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymax=mse_alpha[argmin])
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_100_MSE_over_alphas_validated_PLM.pdf')

In [ ]:
mse_alpha

## Experiment 6. Weak models revisited with validation stop criterion

In [ ]:
number_of_tasks = 200
number_of_points_per_task = 20
number_of_points_per_task_for_validation = 10
dataset_name = 'artificial_sine_medium'
validation_dataset_name = 'artificial_sine_medium_validation'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
for i in range(number_of_tasks):
    a = 0.1 + rng.random() * (2.0 - 0.1)
    a_array.append(a)
    b = rng.random() * 2 * np.pi
    b_array.append(b)
    x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
    x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
    y_train = a * np.sin(x_train + b)
    y_val = a * np.sin(x_val + b)
    X_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = x_train.copy()
    X_sine_val[number_of_points_per_task_for_validation * i : number_of_points_per_task_for_validation * (i + 1)] = x_val.copy()    
    y_sine[number_of_points_per_task * i : number_of_points_per_task * (i + 1)] = y_train.squeeze(1).copy()
    y_sine_val[number_of_points_per_task_for_validation * i : number_of_points_per_task_for_validation * (i + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
for i in range(min(5, number_of_tasks)):
    inds = range(number_of_points_per_task * i, number_of_points_per_task * (i + 1))
    plt.scatter(X_sine[inds], y_sine[inds])

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_medium_validated'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
saves_name = '20_200_medium_with_validation'
for i in range(9, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '/20_200_medium_with_validation', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=mse_alpha[argmin])
plt.tight_layout()
plt.savefig(PLOT_PATH + '/20_200_medium_with_validation.pdf')

### Performance on existing clients

In [ ]:
mse_table = np.empty(shape=(5, models[0].n_workers))

In [ ]:
models[0].change_alpha(0.0)

In [ ]:
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
for i in range(10):
    print(models[i].workers[0].alpha)

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(5):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
mse_alpha.shape

In [ ]:
plt.plot(mse_alpha)

### Performance on a new client

In [ ]:
n_train = 30
n_test = 1000

In [ ]:
rng = default_rng()
a_cn = 0.1 + rng.random() * (2.0 - 0.1)
b_cn = rng.random() * 2 * np.pi
x_train_cn = -5.0 + rng.random((n_train, 1)) * 10.0
y_train_cn = a_cn * np.sin(x_train_cn + b_cn)

In [ ]:
plt.scatter(x_train_cn, y_train_cn)

In [ ]:
control_node = NN_1d_regression(id_node=1000, alpha=0.5, x_train=x_train_cn, y_train=y_train_cn, regularization=None, tolerance=0.1)

In [ ]:
control_node.set_weights(control_node.w_opt)

In [ ]:
y_pred_train = control_node.model(torch.from_numpy(x_train_cn).float()).detach().numpy()

In [ ]:
plt.scatter(x_train_cn, y_pred_train)

In [ ]:
x_test_cn = -5.0 + rng.random((n_test, 1)) * 10.0
y_test_cn = a_cn * np.sin(x_test_cn + b_cn)

In [ ]:
plt.scatter(x_test_cn, y_test_cn)

In [ ]:
mse_array = np.empty(5)

In [ ]:
for i in range(5):
    control_node.alpha = 0.1 * i
    control_node.set_weights(control_node.compute_local(models[i].w_opt_global))
    y_prediction_cn = control_node.model(torch.from_numpy(x_test_cn).float()).detach().numpy()
    mse_array[i] = np.mean((y_prediction_cn - y_test_cn) ** 2)

In [ ]:
mse_array

## Experiment 7. Two distribution case

In [5]:
number_of_tasks = 200
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 20
dataset_name = 'artificial_sine_two_distribution'
validation_dataset_name = 'artificial_sine_two_distribution_validation'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
for j in range(2):
    a = 0.1 + rng.random() * (2.0 - 0.1)
    b = rng.random() * 2 * np.pi
    for i in range(int(number_of_tasks / 2)):
        a_array.append(a)
        b_array.append(b)
        
        x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
        x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
        y_train = a * np.sin(x_train + b)
        y_val = a * np.sin(x_val + b)
        
        ind = i + int(number_of_tasks * j / 2)
        X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
        X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
        y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
        y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_two_distribution_validated'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [6]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [7]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

------------------- alpha = 0.05 --------------------
Computing Lipschitz smoothness constant...
Current L = 102.400000           744/60000 Iterations: fun_value 0.000330 grad_norm 0.015335 fun_value_on_validation 0.0004574
Worker 0 smoothness constant: 0.1
Computing Lipschitz smoothness constant...


KeyboardInterrupt: 

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
saves_name = '50_200_two_distribution_validation'
for i in range(1, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '/50_200_two_distribution_validation', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_200_two_distribution_validation.pdf')

## Experiment 8. Two distribution case, 200-100

In [ ]:
number_of_tasks = 300
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 20
dataset_name = 'artificial_sine_two_distribution_200vs100'
validation_dataset_name = 'artificial_sine_two_distribution_validation_200vs100'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
a_array_old = np.load(DATASET_PATH + 'a_two_distribution_validated.npy')
b_array_old = np.load(DATASET_PATH + 'b_two_distribution_validated.npy')
dist_sizes = [200, 100]
for j in range(2):
    a = a_array_old[100 * j]
    b = b_array_old[100 * j]
    for i in range(dist_sizes[j]):
        a_array.append(a)
        b_array.append(b)
        
        x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
        x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
        y_train = a * np.sin(x_train + b)
        y_val = a * np.sin(x_val + b)
        
        ind = i + dist_sizes[0] * j
        X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
        X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
        y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
        y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_two_distribution_validated_200vs100'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
saves_name = '50_300_two_distribution_validation200vs100'
for i in range(1, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '/50_300_two_distribution_validation200vs100', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_300_two_distribution_validation200vs100.pdf')

## Experiment 9. Two distribution case, 500-100

In [ ]:
number_of_tasks = 600
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 20
dataset_name = 'artificial_sine_two_distribution_500vs100'
validation_dataset_name = 'artificial_sine_two_distribution_validation_500vs100'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
a_array_old = np.load(DATASET_PATH + 'a_two_distribution_validated.npy')
b_array_old = np.load(DATASET_PATH + 'b_two_distribution_validated.npy')
dist_sizes = [500, 100]
for j in range(2):
    a = a_array_old[100 * j]
    b = b_array_old[100 * j]
    for i in range(dist_sizes[j]):
        a_array.append(a)
        b_array.append(b)
        
        x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
        x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
        y_train = a * np.sin(x_train + b)
        y_val = a * np.sin(x_val + b)
        
        ind = i + dist_sizes[0] * j
        X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
        X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
        y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
        y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_two_distribution_validated_500vs100'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
saves_name = '50_600_two_distribution_validation500vs100'
for i in range(1, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '50_600_two_distribution_validation500vs100', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
plt.title('500 - 100')
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_600_two_distribution_validation500vs100.pdf')

## Experiment 10. Two distribution case, 300-100

In [ ]:
number_of_tasks = 400
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 20
dataset_name = 'artificial_sine_two_distribution_300vs100'
validation_dataset_name = 'artificial_sine_two_distribution_validation_300vs100'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
a_array_old = np.load(DATASET_PATH + 'a_two_distribution_validated.npy')
b_array_old = np.load(DATASET_PATH + 'b_two_distribution_validated.npy')
dist_sizes = [300, 100]
for j in range(2):
    a = a_array_old[100 * j]
    b = b_array_old[100 * j]
    for i in range(dist_sizes[j]):
        a_array.append(a)
        b_array.append(b)
        
        x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
        x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
        y_train = a * np.sin(x_train + b)
        y_val = a * np.sin(x_val + b)
        
        ind = i + dist_sizes[0] * j
        X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
        X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
        y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
        y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_two_distribution_validated_300vs100'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
saves_name = '50_400_two_distribution_validation300vs100'
for i in range(5, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)
            
            if f_value_ is None:
                raise Exception('None is detected') 

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '50_400_two_distribution_validation300vs100', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
plt.title('300 - 100')
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_400_two_distribution_validation400vs100.pdf')

## Experiment 11. Two distribution case, 1000-100

In [ ]:
number_of_tasks = 1100
number_of_points_per_task = 50
number_of_points_per_task_for_validation = 20
dataset_name = 'artificial_sine_two_distribution_1000vs100'
validation_dataset_name = 'artificial_sine_two_distribution_validation_1000vs100'

In [ ]:
X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))

In [ ]:
rng = default_rng()
a_array = []
b_array = []
a_array_old = np.load(DATASET_PATH + 'a_two_distribution_validated.npy')
b_array_old = np.load(DATASET_PATH + 'b_two_distribution_validated.npy')
dist_sizes = [1000, 100]
for j in range(2):
    a = a_array_old[100 * j]
    b = b_array_old[100 * j]
    for i in range(dist_sizes[j]):
        a_array.append(a)
        b_array.append(b)
        
        x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
        x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
        y_train = a * np.sin(x_train + b)
        y_val = a * np.sin(x_val + b)
        
        ind = i + dist_sizes[0] * j
        X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
        X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
        y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
        y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
a_array = np.array(a_array)
b_array = np.array(b_array)

In [ ]:
ind = 1099
ind_range = list(range(ind * number_of_points_per_task, (ind + 1) * number_of_points_per_task))
plt.scatter(X_sine[ind_range], y_sine[ind_range])

In [ ]:
dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)

In [ ]:
key_word = '_two_distribution_validated_1000vs100'

In [ ]:
np.save(DATASET_PATH + 'a' + key_word + '.npy', a_array)
np.save(DATASET_PATH + 'b' + key_word + '.npy', b_array)

In [ ]:
a_array = np.load(DATASET_PATH + 'a' + key_word + '.npy')
b_array = np.load(DATASET_PATH + 'b' + key_word + '.npy')

### Pure local models

In [ ]:
alpha = 0.05
n_workers = number_of_tasks
exp = 'gd'
max_it = 100

alg = NN_1d_regression
logreg = False

In [ ]:
print('------------------- alpha = {} --------------------'.format(alpha))
model = MasterNode(n_workers, alpha, alg, dataset_name, logreg, True, max_it, tolerance=1e-2, validation=True, validation_dataset_name=validation_dataset_name)
print('Running GD...')
model.run_gd(max_it)

### Training a global model

In [ ]:
models = []
for i in range(11):
    models.append(copy.deepcopy(model))

In [ ]:
alphas = np.linspace(0, 1.0, 11)

In [ ]:
for i in range(1,11):
    models[i].change_alpha(alphas[i])

In [ ]:
models[0].change_alpha(0.0)
models[0].w_opt_global = np.zeros(models[0].d)

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
saves_name = '50_11000_two_distribution_validation1000vs100'

In [ ]:
save(models, saves_name)

In [ ]:
for i in range(1, 11):
    model = models[i]
    min_L = 0.1
    max_L = 0.1
    max_it = 10000
    tol = 1e-2
    max_L_constant = 2 ** 40
    w = copy.deepcopy(models[i-1].w_opt_global)
    grad_norm = None
    min_f_value = float('Inf')

    try:
        for it in range(max_it):
            grad = model.grad(w)
            L = min_L
            curr_fun_value = model.fun_value(w)

            while True:
                if L > max_L_constant: # if L becomes too large, jump to another random point w
                    w = np.random.randn(model.d)
                    grad = model.grad(w)
                    L = min_L
                    curr_fun_value = model.fun_value(w)

                print('Current L = {:f}'.format(L), end='\r')

                f_value_ = model.fun_value(w - grad / L)
                if curr_fun_value - f_value_ > 0:
                    break
                L *= 2.0

            w -= grad / L
            grad_norm = la.norm(grad)
            
            if f_value_ is None:
                raise Exception('None is detected') 

            if f_value_ < min_f_value:
                min_f_value = f_value_
                model.w_opt_global = copy.deepcopy(w)

            if max_L < L:
                max_L = L
            print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
            if grad_norm < tol and f_value_ < tol ** 2:
                save(models, saves_name)
                break
    except KeyboardInterrupt:
        print('')
        print(min_f_value)
    else:
        save(models, saves_name)
        print('')
        print('Unknown problem')

In [ ]:
with open(MODELS_PATH + '50_11000_two_distribution_validation1000vs100', 'rb') as file:
    models = pickle.load(file)['models']

In [ ]:
for i in range(11):
    model = models[i]
    print(model.fun_value(model.w_opt_global))

In [ ]:
mse_table = np.empty(shape=(11, models[0].n_workers))

In [ ]:
n_test = 2000
rng = default_rng()
for i in range(models[0].n_workers):
    x_test = -5.0 + rng.random((n_test, 1)) * 10
    y_test = a_array[i] * np.sin(x_test + b_array[i])
    for j in range(11):
        worker = models[j].workers[i]
        worker.set_weights(worker.compute_local(models[j].w_opt_global))
        y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
        mse = np.mean((y_pred - y_test) ** 2).item()
        mse_table[j][i] = copy.copy(mse)

In [ ]:
mse_alpha = np.mean(mse_table, axis=1)

In [ ]:
alphas = np.linspace(0, 1.0, 11)
argmin = np.argmin(mse_alpha)
alpha_min = alphas[argmin]

In [ ]:
mse_alpha

In [ ]:
argmin

In [ ]:
mse_alpha[argmin]

In [ ]:
plt.plot(alphas, mse_alpha, marker='o')
plt.yscale('log')
plt.xlabel('alpha')
plt.ylabel('Average MSE over clients')
plt.xticks(alphas)
plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
plt.title('300 - 100')
plt.tight_layout()
plt.savefig(PLOT_PATH + '/50_1100_two_distribution_validation1000vs100.pdf')